In [2]:
import git
# Clone the repository
repo_url = 'https://github.com/mobile-insight/mobileinsight-core.git'
repo_dir = r'C:\Users\bhull\Desktop\CS 219\mobileinsight'
git.Repo.clone_from(repo_url, repo_dir)

In [ ]:
# OBSOLETE

import openai
import os
openai.api_key = 'INSERT_HERE'

def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response['data'][0]['embedding']

def get_text_from_files(repo_dir):
    text_data = {}
    for root, _, files in os.walk(repo_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    text_data[file_path] = f.read()
            except UnicodeDecodeError:
                try:
                    with open(file_path, 'r', encoding='latin-1') as f:
                        text_data[file_path] = f.read()
                except Exception as e:
                    print(f"Could not read file {file_path}: {e}")
    return text_data

repo_text_data = get_text_from_files(repo_dir)
embeddings = {path: generate_embeddings(text) for path, text in repo_text_data.items()}


In [8]:
import os
from typing import Optional
from tenacity import retry, stop_after_attempt, wait_exponential
import shutil
import requests

import chromadb
from chromadb.config import Settings
# from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import GPT4AllEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain import hub
from langchain_core.output_parsers import StrOutputParser


CHROMA_DB_DIRECTORY='db'
DOCUMENT_SOURCE_DIRECTORY='INSERT_HERE'
CHROMA_SETTINGS = Settings(
    chroma_db_impl='duckdb+parquet',
    persist_directory=CHROMA_DB_DIRECTORY,
    anonymized_telemetry=False
)
TARGET_SOURCE_CHUNKS=4
CHUNK_SIZE=500
CHUNK_OVERLAP=50
HIDE_SOURCE_DOCUMENTS=False

class MyKnowledgeBase:
    def __init__(self, source_folder_path: str) -> None:
        """
        Loads pdf and creates a Knowledge base using the Chroma
        vector DB.
        Args:
            pdf_source_folder_path (str): The source folder containing 
            all the pdf documents
        """
        self.source_folder_path = source_folder_path

    def convert_files_to_txt(self):
        txt_folder_path = os.path.join(self.source_folder_path, 'txt_files')
        os.makedirs(txt_folder_path, exist_ok=True)

        for root, _, files in os.walk(self.source_folder_path):
            for file in files:
                if not file.endswith('.txt'):
                    file_path = os.path.join(root, file)
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            content = f.read()
                        txt_file_path = os.path.join(txt_folder_path, file + '.txt')
                        with open(txt_file_path, 'w', encoding='utf-8') as f:
                            f.write(content)
                    except Exception as e:
                        print(f"Could not convert file {file_path}: {e}")
        
        return txt_folder_path

    def load_txts(self, folder_path):
        loader = DirectoryLoader(folder_path)
        loaded_pdfs = loader.load()
        return loaded_pdfs

    def split_documents(
        self,
        loaded_docs,
    ):
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP,
        )
        chunked_docs = splitter.split_documents(loaded_docs)
        return chunked_docs

    def convert_document_to_embeddings(self, chunked_docs, embedder):
 
        db = Chroma.from_documents(chunked_docs, embedder)
        return db

    def initiate_document_injection_pipeline(self):
        txt_folder_path = self.convert_files_to_txt()
        loaded_txts = self.load_txts(txt_folder_path)
        chunked_documents = self.split_documents(loaded_docs=loaded_txts)
        
        print("=> PDF loading and chunking done.")

        vector_db = self.convert_document_to_embeddings(
            chunked_docs=chunked_documents, embedder=OpenAIEmbeddings()
        )

        print("=> vector db initialised and created.")
        print("All done")
        return vector_db

In [10]:
# kb is knowledge base
kb = MyKnowledgeBase(
        source_folder_path=r'C:\Users\bhull\Desktop\CS 219\mobileinsight'
)

vector_db = kb.initiate_document_injection_pipeline()

Could not convert file C:\Users\bhull\Desktop\CS 219\mobileinsight\.git\index: 'utf-8' codec can't decode byte 0x8d in position 15: invalid start byte
Could not convert file C:\Users\bhull\Desktop\CS 219\mobileinsight\.git\objects\pack\pack-eea9c2079c191527f2c02b802f89fc9148ce2de1.idx: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Could not convert file C:\Users\bhull\Desktop\CS 219\mobileinsight\.git\objects\pack\pack-eea9c2079c191527f2c02b802f89fc9148ce2de1.pack: 'utf-8' codec can't decode bytes in position 11-12: invalid continuation byte
Could not convert file C:\Users\bhull\Desktop\CS 219\mobileinsight\.git\objects\pack\pack-eea9c2079c191527f2c02b802f89fc9148ce2de1.rev: 'utf-8' codec can't decode byte 0x82 in position 19: invalid start byte
Could not convert file C:\Users\bhull\Desktop\CS 219\mobileinsight\gui\icons\about.png: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte
Could not convert file C:\Users\bhull\Desktop\CS 219\mobilei

In [31]:
# ZERO-SHOT RAG

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key="")

template = """You are writing python scripts for the open-source project Mobileinsight.
You will write scripts to analyze operational traces collected from 5G devices. Given a natural language
prompt or problem, you will write a python-script that analyzes 5G logs based on the problem.
You are provided with the relevant source code directly from the Mobileinsight repository for the prompt that I will be asking, which you should utilize for my prompt!
NOTE: DO NOT WRITE ANY OTHER TEXT IN THE OUTPUT EXCEPT THE CODE. THE OUTPUT SHOULD BE ABLE TO BE RUN OUT OF THE BOX.
You need to use the relevant source code on top of your existing knowledge of Mobileinsight to answer the prompt. 
Please do NOT leave out any of the implementations (i.e fully complete all methods), the code should run out of the box!
Here is the relevant source code:
{context}

Prompt: {question}
"""
custom_rag_prompt = PromptTemplate.from_template(template)
retriever = kb.return_retriever_from_persistant_vector_db(vector_db)
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [33]:
with open(r"C:\Users\bhull\Desktop\CS 219\rag_prompt.txt", "r") as file:
    question = file.read().replace("\n", " ") # replace newline with space

analyzer_code = str(rag_chain.invoke(question))

analyzer_code = analyzer_code.replace("```python", "", 1)
analyzer_code = analyzer_code.replace("```", "", 1)

file_name= "rag_attach_stats_zero_shot_output.py"

with open(file_name, 'w') as file:
    file.write(analyzer_code)

print(f"Python code has been saved to {file_name}")

Python code has been saved to rag_attach_stats_zero_shot_output.py


In [34]:
# IN-CONTEXT RAG
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key="")

# SAME AS ABOVE TEMPLATE (ZERO-SHOT) BUT WITH EXAMPLE PROMPT AND EXPECTED OUTPUT USED IN BASELINE IN-CONTEXT LLM
template = """You are writing python scripts for the open-source project Mobileinsight.
You will write scripts to analyze operational traces collected from 5G devices. Given a natural language
prompt or problem, you will write a python-script that analyzes 5G logs based on the problem.
You are provided with the relevant source code directly from the Mobileinsight repository for the prompt that I will be asking, which you should utilize for my prompt!
NOTE: DO NOT WRITE ANY OTHER TEXT IN THE OUTPUT EXCEPT THE CODE. THE OUTPUT SHOULD BE ABLE TO BE RUN OUT OF THE BOX.
Here is an example prompt:\
I want you to define a class `myAnalyzer` that inherits from a base `Analyzer` class, and returns intervals for control plane service requests:\
    \
    1. Class Definition: `myAnalyzer`\
    This class extends from a base `Analyzer` class. Through `set_source`, it configures which signaling messages to read by enabling logs for incoming and outgoing NAS EMM packets and RRC OTA packets.\
    The `__msg_callback` function processes messages based on their type:\
    - NAS EMM Incoming Packets: Parses XML data to capture 'Attach accept' messages and logs them.\
    - RRC OTA Packets: Logs 'RRC release' and 'RRC connection request' messages based on the message length.\
    - NAS EMM Outgoing Packets: Captures 'Control plane service request' messages from the outgoing packets.\
    \
    2. Analysis Function: `my_analysis`\
    Initialize an `OfflineReplayer`and set the path for the input data. Configure the source with an instance of `myAnalyzer` and runs the source to process the data. You should include error handling to manage exceptions during execution.\
    \
    3. Execution Logic\
    Upon execution, which is triggered via command-line with an input file path, the script processes the log file through the configured analyzer. It then handles the output by filtering out 'Control plane service request' messages that \
    immediately follow an 'Attach accept' to avoid double counting in specific scenarios. Subsequently, it calculates the time intervals between 'RRC release' and the next 'RRC connection request' for each 'Control plane service request'. \
    These intervals are logged to a CSV file named 'interval_stats.csv'.\
    Here is the expected output:\
    #!/usr/bin/python\
    \
    import sys\
    import csv\
    \
    from mobile_insight.monitor import OfflineReplayer\
    \
    __all__ = [\"myAnalyzer\"]\
    \
    try:\
        import xml.etree.cElementTree as ET\
    except ImportError:\
        import xml.etree.ElementTree as ET\
    from mobile_insight.analyzer.analyzer import *\
    \
    class myAnalyzer(Analyzer):\
        def __init__(self):\
            Analyzer.__init__(self)\
            self.add_source_callback(self.__msg_callback)\
            self.new_attach = False\
            self.messages = []\
    \
        def set_source(self, source):\
            \"""\
            Set the trace source. Enable the cellular signaling messages\
    \
            :param source: the trace source (collector).\
            \"""\
            Analyzer.set_source(self, source)\
            #source.enable_log(\"LTE_NAS_ESM_OTA_Incoming_Packet\")\
            #source.enable_log(\"LTE_NAS_ESM_OTA_Outgoing_Packet\")\
            source.enable_log(\"LTE_NAS_EMM_OTA_Incoming_Packet\")\
            source.enable_log(\"LTE_NAS_EMM_OTA_Outgoing_Packet\")\
            source.enable_log(\"LTE_RRC_OTA_Packet\")\
            # source.enable_log_all()    \
    \
        def __msg_callback(self, msg):\
            if msg.type_id == \"LTE_NAS_EMM_OTA_Incoming_Packet\":\
                data = msg.data.decode()\
                if 'Msg' in data.keys():\
                    log_xml = ET.XML(data['Msg'])\
                    #print('x')\
                else:\
                    return\
                xml_msg = Event(msg.timestamp, msg.type_id, log_xml)\
                for field in xml_msg.data.iter('field'):\
                    if field.get('name') != None and 'nas_eps.nas_msg' in field.get('name'):\
                        if field.get('showname') == 'NAS EPS Mobility Management Message Type: Attach accept (0x42)':\
                            self.messages.append(('attach_accept', data[\"timestamp\"]))\
            elif msg.type_id == \"LTE_RRC_OTA_Packet\":\
                data = msg.data.decode()\
                if \"log_msg_len\" in data and data[\"log_msg_len\"]==33:\
                    self.messages.append(('rrc_release', data[\"timestamp\"]))\
                elif \"log_msg_len\" in data and data[\"log_msg_len\"]==40:\
                    self.messages.append(('rrc_conn_req', data[\"timestamp\"]))\
            elif msg.type_id == \"LTE_NAS_EMM_OTA_Outgoing_Packet\":\
                data = msg.data.decode()\
                if 'Msg' in data.keys():\
                    log_xml = ET.XML(data['Msg'])\
                else:\
                    return\
                xml_msg = Event(msg.timestamp, msg.type_id, log_xml)\
                for field in xml_msg.data.iter('field'):\
                    if field.get('name') != None and 'nas_eps.nas_msg' in field.get('name'):\
                        if field.get('showname') == 'NAS EPS Mobility Management Message Type: Control plane service request (0x4d)':\
                            self.messages.append(('ctrl_pln_svc_req', data[\"timestamp\"]))\
    \
    def my_analysis(input_path):\
    \
        src = OfflineReplayer()\
        src.set_input_path(input_path)\
    \
        analyzer = myAnalyzer()\
        analyzer.set_source(src)\
        try:\
            src.run()\
        except:\
            print('Failed:', input_path)\
            return None\
    \
        return analyzer\
    \
    \
    input_path = sys.argv[1]\
    analyzer = my_analysis(input_path)\
    if analyzer:\
        ignore_next_ctrl_pln_svc_req = False\
        processed_messages = []\
        for message in analyzer.messages:\
            if message[0] == 'attach_accept':\
                ignore_next_ctrl_pln_svc_req = True\
            elif message[0] == 'ctrl_pln_svc_req' and ignore_next_ctrl_pln_svc_req:\
                ignore_next_ctrl_pln_svc_req = False\
            else:\
                processed_messages.append(message)\
                if ignore_next_ctrl_pln_svc_req:\
                    ignore_next_ctrl_pln_svc_req = False  # Reset flag if it was not used\
    \
        # Find and calculate differences\
        for i, msg in enumerate(processed_messages):\
            if msg[0] == 'ctrl_pln_svc_req':\
                # Find the closest rrc_release before it\
                closest_rrc_release = max((m for m in processed_messages[:i] if m[0] == 'rrc_release'), default=None, key=lambda x: x[1])\
                # Find the closest rrc_conn_req after it\
                closest_rrc_conn_req = min((m for m in processed_messages[i+1:] if m[0] == 'rrc_conn_req'), default=None, key=lambda x: x[1])\
                \
                if closest_rrc_release and closest_rrc_conn_req:\
                    # Calculate the difference\
                    difference = closest_rrc_conn_req[1] - closest_rrc_release[1]\
                    with open('interval_stats.csv', 'a') as f:\
                        row = [input_path, difference]\
                        writer = csv.writer(f)\
                        writer.writerow(row)\
You need to use the relevant source code and the example above on top of your existing knowledge of Mobileinsight to answer the prompt. 
Please do NOT leave out any of the implementations (i.e fully complete all methods), the code should run out of the box!
Here is the relevant source code:
{context}

Prompt: {question}
"""
custom_rag_prompt = PromptTemplate.from_template(template)
retriever = kb.return_retriever_from_persistant_vector_db(vector_db)
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [35]:
with open(r"C:\Users\bhull\Desktop\CS 219\rag_prompt.txt", "r") as file:
    question = file.read().replace("\n", " ") # replace newline with space

analyzer_code = str(rag_chain.invoke(question))

analyzer_code = analyzer_code.replace("```python", "", 1)
analyzer_code = analyzer_code.replace("```", "", 1)

file_name= "rag_attach_stats_in_context_output.py"

with open(file_name, 'w') as file:
    file.write(analyzer_code)

print(f"Python code has been saved to {file_name}")

Python code has been saved to rag_attach_stats_in_context_output.py


In [36]:
# FINE-TUNED ZERO-SHOT RAG

llm = ChatOpenAI(model="ft:gpt-3.5-turbo-0125:personal::9Tiw2Bjt", api_key="")

template = """You are writing python scripts for the open-source project Mobileinsight.
You will write scripts to analyze operational traces collected from 5G devices. Given a natural language
prompt or problem, you will write a python-script that analyzes 5G logs based on the problem.
You are provided with the relevant source code directly from the Mobileinsight repository for the prompt that I will be asking, which you should utilize for my prompt!
NOTE: DO NOT WRITE ANY OTHER TEXT IN THE OUTPUT EXCEPT THE CODE. THE OUTPUT SHOULD BE ABLE TO BE RUN OUT OF THE BOX.
You need to use the relevant source code on top of your existing knowledge of Mobileinsight to answer the prompt. 
Please do NOT leave out any of the implementations (i.e fully complete all methods), the code should run out of the box!
Here is the relevant source code:
{context}

Prompt: {question}
"""
custom_rag_prompt = PromptTemplate.from_template(template)
retriever = kb.return_retriever_from_persistant_vector_db(vector_db)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [38]:
with open(r"C:\Users\bhull\Desktop\CS 219\rag_prompt.txt", "r") as file:
    question = file.read().replace("\n", " ") # replace newline with space

analyzer_code = str(rag_chain.invoke(question))

analyzer_code = analyzer_code.replace("```python", "", 1)
analyzer_code = analyzer_code.replace("```", "", 1)

file_name= "finetuned_ZERO_SHOT_attach_stats_output.py"

with open(file_name, 'w') as file:
    file.write(analyzer_code)

print(f"Python code has been saved to {file_name}")

Python code has been saved to finetuned_ZERO_SHOT_attach_stats_in_context_output.py


In [42]:
# FINE-TUNED IN-CONTEXT RAG
llm = ChatOpenAI(model="ft:gpt-3.5-turbo-0125:personal::9Tiw2Bjt", api_key="INSERT_HERE")

template = """You are writing python scripts for the open-source project Mobileinsight.
You will write scripts to analyze operational traces collected from 5G devices. Given a natural language
prompt or problem, you will write a python-script that analyzes 5G logs based on the problem.
You are provided with the relevant source code directly from the Mobileinsight repository for the prompt that I will be asking, which you should utilize for my prompt!
NOTE: DO NOT WRITE ANY OTHER TEXT IN THE OUTPUT EXCEPT THE CODE. THE OUTPUT SHOULD BE ABLE TO BE RUN OUT OF THE BOX.
Here is an example prompt:\
I want you to define a class `myAnalyzer` that inherits from a base `Analyzer` class, and returns intervals for control plane service requests:\
    \
    1. Class Definition: `myAnalyzer`\
    This class extends from a base `Analyzer` class. Through `set_source`, it configures which signaling messages to read by enabling logs for incoming and outgoing NAS EMM packets and RRC OTA packets.\
    The `__msg_callback` function processes messages based on their type:\
    - NAS EMM Incoming Packets: Parses XML data to capture 'Attach accept' messages and logs them.\
    - RRC OTA Packets: Logs 'RRC release' and 'RRC connection request' messages based on the message length.\
    - NAS EMM Outgoing Packets: Captures 'Control plane service request' messages from the outgoing packets.\
    \
    2. Analysis Function: `my_analysis`\
    Initialize an `OfflineReplayer`and set the path for the input data. Configure the source with an instance of `myAnalyzer` and runs the source to process the data. You should include error handling to manage exceptions during execution.\
    \
    3. Execution Logic\
    Upon execution, which is triggered via command-line with an input file path, the script processes the log file through the configured analyzer. It then handles the output by filtering out 'Control plane service request' messages that \
    immediately follow an 'Attach accept' to avoid double counting in specific scenarios. Subsequently, it calculates the time intervals between 'RRC release' and the next 'RRC connection request' for each 'Control plane service request'. \
    These intervals are logged to a CSV file named 'interval_stats.csv'.\
    Here is the expected output:\
    #!/usr/bin/python\
    \
    import sys\
    import csv\
    \
    from mobile_insight.monitor import OfflineReplayer\
    \
    __all__ = [\"myAnalyzer\"]\
    \
    try:\
        import xml.etree.cElementTree as ET\
    except ImportError:\
        import xml.etree.ElementTree as ET\
    from mobile_insight.analyzer.analyzer import *\
    \
    class myAnalyzer(Analyzer):\
        def __init__(self):\
            Analyzer.__init__(self)\
            self.add_source_callback(self.__msg_callback)\
            self.new_attach = False\
            self.messages = []\
    \
        def set_source(self, source):\
            \"""\
            Set the trace source. Enable the cellular signaling messages\
    \
            :param source: the trace source (collector).\
            \"""\
            Analyzer.set_source(self, source)\
            #source.enable_log(\"LTE_NAS_ESM_OTA_Incoming_Packet\")\
            #source.enable_log(\"LTE_NAS_ESM_OTA_Outgoing_Packet\")\
            source.enable_log(\"LTE_NAS_EMM_OTA_Incoming_Packet\")\
            source.enable_log(\"LTE_NAS_EMM_OTA_Outgoing_Packet\")\
            source.enable_log(\"LTE_RRC_OTA_Packet\")\
            # source.enable_log_all()    \
    \
        def __msg_callback(self, msg):\
            if msg.type_id == \"LTE_NAS_EMM_OTA_Incoming_Packet\":\
                data = msg.data.decode()\
                if 'Msg' in data.keys():\
                    log_xml = ET.XML(data['Msg'])\
                    #print('x')\
                else:\
                    return\
                xml_msg = Event(msg.timestamp, msg.type_id, log_xml)\
                for field in xml_msg.data.iter('field'):\
                    if field.get('name') != None and 'nas_eps.nas_msg' in field.get('name'):\
                        if field.get('showname') == 'NAS EPS Mobility Management Message Type: Attach accept (0x42)':\
                            self.messages.append(('attach_accept', data[\"timestamp\"]))\
            elif msg.type_id == \"LTE_RRC_OTA_Packet\":\
                data = msg.data.decode()\
                if \"log_msg_len\" in data and data[\"log_msg_len\"]==33:\
                    self.messages.append(('rrc_release', data[\"timestamp\"]))\
                elif \"log_msg_len\" in data and data[\"log_msg_len\"]==40:\
                    self.messages.append(('rrc_conn_req', data[\"timestamp\"]))\
            elif msg.type_id == \"LTE_NAS_EMM_OTA_Outgoing_Packet\":\
                data = msg.data.decode()\
                if 'Msg' in data.keys():\
                    log_xml = ET.XML(data['Msg'])\
                else:\
                    return\
                xml_msg = Event(msg.timestamp, msg.type_id, log_xml)\
                for field in xml_msg.data.iter('field'):\
                    if field.get('name') != None and 'nas_eps.nas_msg' in field.get('name'):\
                        if field.get('showname') == 'NAS EPS Mobility Management Message Type: Control plane service request (0x4d)':\
                            self.messages.append(('ctrl_pln_svc_req', data[\"timestamp\"]))\
    \
    def my_analysis(input_path):\
    \
        src = OfflineReplayer()\
        src.set_input_path(input_path)\
    \
        analyzer = myAnalyzer()\
        analyzer.set_source(src)\
        try:\
            src.run()\
        except:\
            print('Failed:', input_path)\
            return None\
    \
        return analyzer\
    \
    \
    input_path = sys.argv[1]\
    analyzer = my_analysis(input_path)\
    if analyzer:\
        ignore_next_ctrl_pln_svc_req = False\
        processed_messages = []\
        for message in analyzer.messages:\
            if message[0] == 'attach_accept':\
                ignore_next_ctrl_pln_svc_req = True\
            elif message[0] == 'ctrl_pln_svc_req' and ignore_next_ctrl_pln_svc_req:\
                ignore_next_ctrl_pln_svc_req = False\
            else:\
                processed_messages.append(message)\
                if ignore_next_ctrl_pln_svc_req:\
                    ignore_next_ctrl_pln_svc_req = False  # Reset flag if it was not used\
    \
        # Find and calculate differences\
        for i, msg in enumerate(processed_messages):\
            if msg[0] == 'ctrl_pln_svc_req':\
                # Find the closest rrc_release before it\
                closest_rrc_release = max((m for m in processed_messages[:i] if m[0] == 'rrc_release'), default=None, key=lambda x: x[1])\
                # Find the closest rrc_conn_req after it\
                closest_rrc_conn_req = min((m for m in processed_messages[i+1:] if m[0] == 'rrc_conn_req'), default=None, key=lambda x: x[1])\
                \
                if closest_rrc_release and closest_rrc_conn_req:\
                    # Calculate the difference\
                    difference = closest_rrc_conn_req[1] - closest_rrc_release[1]\
                    with open('interval_stats.csv', 'a') as f:\
                        row = [input_path, difference]\
                        writer = csv.writer(f)\
                        writer.writerow(row)\
You need to use the relevant source code and the example above on top of your existing knowledge of Mobileinsight to answer the prompt. 
Please do NOT leave out any of the implementations (i.e fully complete all methods), the code should run out of the box!
Here is the relevant source code:
{context}

Prompt: {question}
"""
custom_rag_prompt = PromptTemplate.from_template(template)
retriever = kb.return_retriever_from_persistant_vector_db(vector_db)
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [41]:
with open(r"C:\Users\bhull\Desktop\CS 219\rag_prompt.txt", "r") as file:
    question = file.read().replace("\n", " ") # replace newline with space

analyzer_code = str(rag_chain.invoke(question))

analyzer_code = analyzer_code.replace("```python", "", 1)
analyzer_code = analyzer_code.replace("```", "", 1)

file_name= "finetuned_IN_CONTEXT_RAG_attach_stats_output.py"

with open(file_name, 'w') as file:
    file.write(analyzer_code)

print(f"Python code has been saved to {file_name}")

Python code has been saved to finetuned_IN_CONTEXT_RAG_attach_stats_in_context_output.py


In [45]:
# SERVICE REQ STATS PROMPT

with open(r"C:\Users\bhull\Desktop\CS 219\testcases\service_req_stats\input.txt", "r") as file:
    question = file.read().replace("\n", " ") # replace newline with space

analyzer_code = str(rag_chain.invoke(question))

analyzer_code = analyzer_code.replace("```python", "", 1)
analyzer_code = analyzer_code.replace("```", "", 1)

file_name= "finetuned_IN_CONTEXT_RAG_service_req_stats_output.py"

with open(file_name, 'w') as file:
    file.write(analyzer_code)

print(f"Python code has been saved to {file_name}")

Python code has been saved to finetuned_IN_CONTEXT_RAG_service_req_stats_output.py
